Import the libraries needed

In [80]:
import pandas as pd
#from pandas.compat import StringIO
from pandas.tseries.offsets import DateOffset

from datetime import datetime
#ateparse = lambda x: datetime.strptime(x, '%Y-%m-%d')
#dateparse = lambda x: datetime.strptime(x '%Y-%m-%d')

import psutil




import numpy as np
import os
import pyodbc
pyodbc.drivers()

import csv

from sklearn.preprocessing import StandardScaler
import seaborn as sns

pd.options.display.max_rows = 1000

# For the Yahoo Finance api
from pandas_datareader import data as pdr
import yfinance as yf
yf.pdr_override() 


### Database connections and SQL to retrieve the various data needed.

In [81]:
SQL_server = 'WINPRDAI1572\CLESQLDW'
Reporting_database = 'CLE_Reporting'

pd.set_option('max_row', None)
#pd.options.display.max_rows = 1000

# Set a value for the home folder.
home_folder = "."

# Set values for the various paths.
input_path = home_folder + "\data"

print('==========================')
now = datetime.now()
print(now)
print('==========================')
print(psutil.virtual_memory())
print('==========================')
os.path.abspath(os.getcwd())

2024-07-20 16:21:27.506473
svmem(total=16756752384, available=3975372800, percent=76.3, used=12781379584, free=3975372800)


'C:\\Users\\m033\\OneDrive - GWLE\\Fund_Switch_Thesis\\Pre_Synthesisation'

# 1. Read back the population with switches data.

In [82]:
# Read back the comma separated values file.

# giving directory name
filename = input_path + '\Full_Population_With_Switch_Flags.csv'

#import CSV file and specify dtype of each column
# https://www.statology.org/pandas-read-csv-dtype/
df_Horizon = pd.read_csv(filename,dtype = {'ContractNumber': str,'ProdCat': str,'ProdCode': str,'PolicyStatus': str,'Switch_Flag': int})
# Set all dates to proper dates
df_Horizon['SnapshotDt'] = pd.to_datetime(df_Horizon['SnapshotDt'])
df_Horizon['CoverEndDt'] = pd.to_datetime(df_Horizon['CoverEndDt'])

print('==========================')
now = datetime.now()
print(now)
print('==========================')
print(psutil.virtual_memory())
print('==========================')

print(df_Horizon.dtypes)
df_Horizon.head(5)

2024-07-20 16:21:38.409053
svmem(total=16756752384, available=3347075072, percent=80.0, used=13409677312, free=3347075072)
ContractNumber            object
ProdCat                   object
ProdCode                  object
PolicyStatus              object
CoverEndDt        datetime64[ns]
SnapshotDt        datetime64[ns]
Switch_Flag                int32
dtype: object


,ContractNumber,ProdCat,ProdCode,PolicyStatus,CoverEndDt,SnapshotDt,Switch_Flag
0,6074160W,XRPP,XRPP_2005V1,I,2028-12-01,2020-01-31,0
1,6074937L,XRPP,XRPP_2005V1,I,2033-12-01,2020-01-31,0
2,6074962M,XRPP,XRPP_2005V1,I,2042-12-01,2020-01-31,0
3,6075850C,XRPP,XRPP_2005V1,F,2034-12-01,2020-01-31,0
4,6100638P,XRPP,XRPP_2005V1,I,2046-04-01,2020-01-31,0


# 2. Get supplementary policy features. 

  -  Add policy level features.
  -  Tax
  -  Asset related features
  -  Indexation and premium.
  -  Broker and Commission 
  -  Client data
  -  customer Market segmentation


## 2.1 Tax, Fee and Risk features.
### All tax and Fee features wre analysed and only one Risk feature remains.

In [83]:
# Read back the comma separated values file.

# giving directory name
filename = input_path + '\Tax_Fee_Risk_data.csv'

#df_Tax_Fee_Risk_data = pd.read_csv(filename, delimiter=',',low_memory=False, header=0)
df_Tax_Fee_Risk_data = pd.read_csv(filename,dtype = {'ContractNumber': str,'Policy_Fee_Type_Desc': str})
# Set all dates to proper dates


print('==========================')
now = datetime.now()
print(now)
print('==========================')
print(psutil.virtual_memory())
print('==========================')

print(df_Tax_Fee_Risk_data.dtypes)
df_Tax_Fee_Risk_data.head(5)

2024-07-20 16:21:38.598804
svmem(total=16756752384, available=3339268096, percent=80.1, used=13417484288, free=3339268096)
ContractNumber          object
Policy_Fee_Type_Desc    object
dtype: object


,ContractNumber,Policy_Fee_Type_Desc
0,6074160W,Ein
1,6074937L,Ein
2,6074962M,Ein
3,6075850C,Ein
4,6100638P,Ein


## Merge in the Tax, Fee and risk data.

In [84]:
df_Horizon_All_Features = pd.merge(df_Horizon, df_Tax_Fee_Risk_data, left_on = ['ContractNumber'], right_on = ['ContractNumber'] , how = 'left')
del df_Tax_Fee_Risk_data
del df_Horizon

print('==========================')
now = datetime.now()
print(now)
print('==========================')
print(psutil.virtual_memory())
print('==========================')

%who_ls DataFrame 

df_Horizon_All_Features.head(5)

2024-07-20 16:21:41.601714
svmem(total=16756752384, available=2937393152, percent=82.5, used=13819359232, free=2937393152)


,ContractNumber,ProdCat,ProdCode,PolicyStatus,CoverEndDt,SnapshotDt,Switch_Flag,Policy_Fee_Type_Desc
0,6074160W,XRPP,XRPP_2005V1,I,2028-12-01,2020-01-31,0,Ein
1,6074937L,XRPP,XRPP_2005V1,I,2033-12-01,2020-01-31,0,Ein
2,6074962M,XRPP,XRPP_2005V1,I,2042-12-01,2020-01-31,0,Ein
3,6075850C,XRPP,XRPP_2005V1,F,2034-12-01,2020-01-31,0,Ein
4,6100638P,XRPP,XRPP_2005V1,I,2046-04-01,2020-01-31,0,Ein


## 2.2 Asset features.
### All asset features were analysed and only one feature remains.

In [85]:
# Read back the comma separated values file.

# giving directory name
filename = input_path + '\Asset_data.csv'

df_Asset = pd.read_csv(filename,dtype = {'ContractNumber': str,'PricingVariant': int})
# Set all dates to proper dates

print('==========================')
now = datetime.now()
print(now)
print('==========================')
print(psutil.virtual_memory())
print('==========================')

print(df_Asset.dtypes)
df_Asset.head(5)

2024-07-20 16:21:41.675333
svmem(total=16756752384, available=2937233408, percent=82.5, used=13819518976, free=2937233408)
ContractNumber    object
PricingVariant     int32
dtype: object


,ContractNumber,PricingVariant
0,6215812A,4015
1,6216004M,4015
2,6216180E,4015
3,6216746P,4015
4,6217141W,4015


## Merge in the asset data.

In [86]:
df_Horizon_All_Features = pd.merge(df_Horizon_All_Features, df_Asset, left_on = ['ContractNumber'], right_on = ['ContractNumber'] , how = 'left')

df_Horizon_All_Features = df_Horizon_All_Features.rename(columns={'Policy_Fee_Type_Desc': 'PolicyFeeType'})

del df_Asset

print('==========================')
now = datetime.now()
print(now)
print('==========================')
print(psutil.virtual_memory())
print('==========================')
print(df_Horizon_All_Features.dtypes)

%who_ls DataFrame 


df_Horizon_All_Features.head(5)

2024-07-20 16:21:45.058831
svmem(total=16756752384, available=2520760320, percent=85.0, used=14235992064, free=2520760320)
ContractNumber            object
ProdCat                   object
ProdCode                  object
PolicyStatus              object
CoverEndDt        datetime64[ns]
SnapshotDt        datetime64[ns]
Switch_Flag                int32
PolicyFeeType             object
PricingVariant           float64
dtype: object


,ContractNumber,ProdCat,ProdCode,PolicyStatus,CoverEndDt,SnapshotDt,Switch_Flag,PolicyFeeType,PricingVariant
0,6074160W,XRPP,XRPP_2005V1,I,2028-12-01,2020-01-31,0,Ein,NaN
1,6074937L,XRPP,XRPP_2005V1,I,2033-12-01,2020-01-31,0,Ein,NaN
2,6074962M,XRPP,XRPP_2005V1,I,2042-12-01,2020-01-31,0,Ein,NaN
3,6075850C,XRPP,XRPP_2005V1,F,2034-12-01,2020-01-31,0,Ein,NaN
4,6100638P,XRPP,XRPP_2005V1,I,2046-04-01,2020-01-31,0,Ein,NaN


## 2.3 Indexation and premium features.
### All Indexation and premium features were analysed and??? only one feature remains.

In [87]:
# Read back the comma separated values file.

# giving directory name
filename = input_path + '\Premium_data.csv'

df_premium = pd.read_csv(filename,dtype = {'ContractNumber': str,
                                           'PayFreq': int},low_memory=False, header=0)
# Set all dates to proper dates
df_premium['AnniversaryDt'] = pd.to_datetime(df_premium['AnniversaryDt'])
df_premium['IndexationStartDt'] = pd.to_datetime(df_premium['IndexationStartDt'])
df_premium['PremHolStartDt'] = pd.to_datetime(df_premium['PremHolStartDt'])
df_premium['PremHolEndDt'] = pd.to_datetime(df_premium['PremHolEndDt'])


print('==========================')
now = datetime.now()
print(now)
print('==========================')
print(psutil.virtual_memory())
print('==========================')

print(df_premium.dtypes)
df_premium.head(5)

2024-07-20 16:21:45.391632
svmem(total=16756752384, available=2504441856, percent=85.1, used=14252310528, free=2504441856)
Contract_Number              object
PayFreq                       int32
AnniversaryDt        datetime64[ns]
IndexationPercent             int64
IndexationStartDt    datetime64[ns]
PremHolStartDt       datetime64[ns]
PremHolEndDt         datetime64[ns]
Premium                     float64
PremiumCalcMethod            object
dtype: object


,Contract_Number,PayFreq,AnniversaryDt,IndexationPercent,IndexationStartDt,PremHolStartDt,PremHolEndDt,Premium,PremiumCalcMethod
0,6074160W,12,2007-12-01,0,2007-12-01,NaT,NaT,221.40,Beitragsvorgabe
1,6074937L,12,2006-12-01,0,2006-12-01,NaT,NaT,157.97,Beitragsvorgabe
2,6074962M,12,2006-12-01,0,2006-12-01,NaT,NaT,157.97,Beitragsvorgabe
3,6075850C,12,2006-12-01,0,2006-12-01,NaT,NaT,0.00,Beitragsvorgabe
4,6100638P,12,2007-04-01,0,2007-04-01,NaT,NaT,137.98,Beitragsvorgabe


## Merge in the Premium and Indexation data.

In [88]:
df_premium = df_premium.rename(columns={'Contract_Number': 'ContractNumber'})

df_Horizon_All_Features = pd.merge(df_Horizon_All_Features, df_premium, left_on = ['ContractNumber'], right_on = ['ContractNumber'] , how = 'left')

del df_premium

print('==========================')
now = datetime.now()
print(now)
print('==========================')
print(psutil.virtual_memory())
print('==========================')
print(df_Horizon_All_Features.dtypes)

%who_ls DataFrame 

df_Horizon_All_Features.head(5)

2024-07-20 16:21:48.720884
svmem(total=16756752384, available=1681010688, percent=90.0, used=15075741696, free=1681010688)
ContractNumber               object
ProdCat                      object
ProdCode                     object
PolicyStatus                 object
CoverEndDt           datetime64[ns]
SnapshotDt           datetime64[ns]
Switch_Flag                   int32
PolicyFeeType                object
PricingVariant              float64
PayFreq                       int32
AnniversaryDt        datetime64[ns]
IndexationPercent             int64
IndexationStartDt    datetime64[ns]
PremHolStartDt       datetime64[ns]
PremHolEndDt         datetime64[ns]
Premium                     float64
PremiumCalcMethod            object
dtype: object


,ContractNumber,ProdCat,ProdCode,PolicyStatus,CoverEndDt,SnapshotDt,Switch_Flag,PolicyFeeType,PricingVariant,PayFreq,AnniversaryDt,IndexationPercent,IndexationStartDt,PremHolStartDt,PremHolEndDt,Premium,PremiumCalcMethod
0,6074160W,XRPP,XRPP_2005V1,I,2028-12-01,2020-01-31,0,Ein,NaN,12,2007-12-01,0,2007-12-01,NaT,NaT,221.40,Beitragsvorgabe
1,6074937L,XRPP,XRPP_2005V1,I,2033-12-01,2020-01-31,0,Ein,NaN,12,2006-12-01,0,2006-12-01,NaT,NaT,157.97,Beitragsvorgabe
2,6074962M,XRPP,XRPP_2005V1,I,2042-12-01,2020-01-31,0,Ein,NaN,12,2006-12-01,0,2006-12-01,NaT,NaT,157.97,Beitragsvorgabe
3,6075850C,XRPP,XRPP_2005V1,F,2034-12-01,2020-01-31,0,Ein,NaN,12,2006-12-01,0,2006-12-01,NaT,NaT,0.00,Beitragsvorgabe
4,6100638P,XRPP,XRPP_2005V1,I,2046-04-01,2020-01-31,0,Ein,NaN,12,2007-04-01,0,2007-04-01,NaT,NaT,137.98,Beitragsvorgabe


## 2.4  Broker and Commission.
### All features were analysed and all remains.

In [89]:
# Read back the comma separated values file.

# giving directory name
filename = input_path + '\Broker_and_Commission_data.csv'

df_Broker_and_Comm = pd.read_csv(filename,dtype = {'ContractNumber': str,
                                                   'ServicingBroker': str,
                                                   'InitialBroker': str,
                                                   'BrokerCategory': str},low_memory=False, header=0)

print('==========================')
now = datetime.now()
print(now)
print('==========================')
print(psutil.virtual_memory())
print('==========================')

print(df_Broker_and_Comm.dtypes)
df_Broker_and_Comm.head(5)

2024-07-20 16:21:48.843592
svmem(total=16756752384, available=1657483264, percent=90.1, used=15099269120, free=1657483264)
ContractNumber      object
ServicingBroker     object
InitialBroker       object
BrokerCategory      object
InitialRate        float64
RenewalRate        float64
dtype: object


,ContractNumber,ServicingBroker,InitialBroker,BrokerCategory,InitialRate,RenewalRate
0,8153732A,1310,1310,B,4.35,1.07
1,6625275M,13860,13860,B,4.30,0.65
2,6751057K,3748,3748,B,4.38,0.96
3,8050943A,5367,5367,D,4.38,0.96
4,8020403N,2802,2802,D,4.38,0.96


## Merge in the Broker and Commission data.

In [90]:
df_Horizon_All_Features = pd.merge(df_Horizon_All_Features, df_Broker_and_Comm, left_on = ['ContractNumber'], right_on = ['ContractNumber'] , how = 'left')

del df_Broker_and_Comm

print('==========================')
now = datetime.now()
print(now)
print('==========================')
print(psutil.virtual_memory())
print('==========================')
print(df_Horizon_All_Features.dtypes)

df_Horizon_All_Features.head(5)

2024-07-20 16:21:52.936790
svmem(total=16756752384, available=1012858880, percent=94.0, used=15743893504, free=1012858880)
ContractNumber               object
ProdCat                      object
ProdCode                     object
PolicyStatus                 object
CoverEndDt           datetime64[ns]
SnapshotDt           datetime64[ns]
Switch_Flag                   int32
PolicyFeeType                object
PricingVariant              float64
PayFreq                       int32
AnniversaryDt        datetime64[ns]
IndexationPercent             int64
IndexationStartDt    datetime64[ns]
PremHolStartDt       datetime64[ns]
PremHolEndDt         datetime64[ns]
Premium                     float64
PremiumCalcMethod            object
ServicingBroker              object
InitialBroker                object
BrokerCategory               object
InitialRate                 float64
RenewalRate                 float64
dtype: object


,ContractNumber,ProdCat,ProdCode,PolicyStatus,CoverEndDt,SnapshotDt,Switch_Flag,PolicyFeeType,PricingVariant,PayFreq,...,IndexationStartDt,PremHolStartDt,PremHolEndDt,Premium,PremiumCalcMethod,ServicingBroker,InitialBroker,BrokerCategory,InitialRate,RenewalRate
0,6074160W,XRPP,XRPP_2005V1,I,2028-12-01,2020-01-31,0,Ein,NaN,12,...,2007-12-01,NaT,NaT,221.40,Beitragsvorgabe,14469,14469,D,4.25,0.625
1,6074937L,XRPP,XRPP_2005V1,I,2033-12-01,2020-01-31,0,Ein,NaN,12,...,2006-12-01,NaT,NaT,157.97,Beitragsvorgabe,NaN,NaN,NaN,NaN,NaN
2,6074962M,XRPP,XRPP_2005V1,I,2042-12-01,2020-01-31,0,Ein,NaN,12,...,2006-12-01,NaT,NaT,157.97,Beitragsvorgabe,7164,7164,D,4.25,0.625
3,6075850C,XRPP,XRPP_2005V1,F,2034-12-01,2020-01-31,0,Ein,NaN,12,...,2006-12-01,NaT,NaT,0.00,Beitragsvorgabe,NaN,NaN,NaN,NaN,NaN
4,6100638P,XRPP,XRPP_2005V1,I,2046-04-01,2020-01-31,0,Ein,NaN,12,...,2007-04-01,NaT,NaT,137.98,Beitragsvorgabe,7164,7164,D,4.25,0.625


In [91]:
%who_ls DataFrame 

['df_Horizon_All_Features']

## 2.5 Client data
### All features were analysed and most remains.

In [99]:
# Read back the comma separated values file.

# giving directory name
filename = input_path + '\Client_data.csv'


df_Client = pd.read_csv(filename,dtype = {'Policy_Number': str,
                                          'VIPType': str,
                                          'Nationality': str,
                                          'IsSmoker': bool,
                                          'PoliticallyExposed': str,
                                          'BirthPlace': str,
                                          'Gender': str,
                                          'MaritalStatus': str,
                                          'AddressCity': str,
                                          'PartnerType': str,
                                          'EUSanctioned': bool,
                                          'AddressDistrict': str,
                                          'Postal_Code_Name': str,
                                          'District': str,
                                          'State': str,
                                          'VN_MILIEU_CODE': str,
                                          'VN_TARGET_GROUP_CODE': str,
                                          'VN_MILIEU_DESCRIPTION': str,
                                          'VP_MILIEU_DESCRIPTION': str},low_memory=False, header=0)
# Set all dates to proper dates
df_Client['RiskCommDt'] = pd.to_datetime(df_Client['RiskCommDt'])
df_Client['DOB'] = pd.to_datetime(df_Client['DOB'])
df_Client['Postal_Code'] = df_Client['Postal_Code'].fillna(0).astype(int)

print('==========================')
now = datetime.now()
print(now)
print('==========================')
print(psutil.virtual_memory())
print('==========================')

print(df_Client.dtypes)
df_Client.head(5)

2024-07-20 16:28:08.884377
svmem(total=16756752384, available=2175918080, percent=87.0, used=14580834304, free=2175918080)
Policy_Number                    object
RiskCommDt               datetime64[ns]
VIPType                          object
Nationality                      object
IsSmoker                           bool
PoliticallyExposed               object
BirthPlace                       object
Gender                           object
MaritalStatus                    object
PensionDisability                 int64
WOPDisability                     int64
AddressCity                      object
DOB                      datetime64[ns]
PartnerType                      object
EUSanctioned                       bool
AddressDistrict                  object
Postal_Code                       int32
Postal_Code_Name                 object
District                         object
State                            object
VN_MILIEU_CODE                   object
VN_TARGET_GROUP_CODE             obje

,Policy_Number,RiskCommDt,VIPType,Nationality,IsSmoker,PoliticallyExposed,BirthPlace,Gender,MaritalStatus,PensionDisability,...,EUSanctioned,AddressDistrict,Postal_Code,Postal_Code_Name,District,State,VN_MILIEU_CODE,VN_TARGET_GROUP_CODE,VN_MILIEU_DESCRIPTION,VP_MILIEU_DESCRIPTION
0,6708910H,2018-03-01,0,DE,False,2,München,2,2,0,...,False,0,83646,Bad T�lz,Landkreis Bad T�lz-Wolfratshausen,Bavaria,NaN,NaN,NaN,NaN
1,6708915W,2018-04-01,0,DE,False,0,Marsberg,1,1,0,...,False,0,34431,Marsberg,Landkreis Hochsauerlandkreis,North Rhine-Westphalia,NaN,NaN,NaN,NaN
2,6708927E,2018-02-01,0,DE,False,2,Mainburg,2,1,0,...,False,0,84106,Volkenschwand,Landkreis Kelheim,Bavaria,10_NOK,BerIntOrientierung,10_NOK,BerIntOrientierung
3,6722995P,2018-05-01,0,GR,False,0,Trikala,1,0,0,...,False,0,81829,Munich,M�nchen,Bavaria,NaN,NaN,NaN,NaN
4,6709066A,2018-03-01,0,DE,False,0,Bühl - Baden,2,1,0,...,False,0,22301,Hamburg,Hamburg,Hamburg,06_NOB,Selbstoptimierer,06_NOB,Selbstoptimierer


## Merge in the Client and address features as well as the Sinus Milieus and postcode features.

In [101]:
df_Client = df_Client.rename(columns={'Policy_Number': 'ContractNumber'})

df_Horizon_All_Features = pd.merge(df_Horizon_All_Features, df_Client, left_on = ['ContractNumber'], right_on = ['ContractNumber'] , how = 'left')

del df_Client

print('==========================')
now = datetime.now()
print(now)
print('==========================')
print(psutil.virtual_memory())
print('==========================')
print(df_Horizon_All_Features.dtypes)

df_Horizon_All_Features.head(5)

2024-07-20 16:29:42.632069
svmem(total=16756752384, available=2207113216, percent=86.8, used=14549639168, free=2207113216)
ContractNumber                   object
ProdCat                          object
ProdCode                         object
PolicyStatus                     object
CoverEndDt               datetime64[ns]
SnapshotDt               datetime64[ns]
Switch_Flag                       int32
PolicyFeeType                    object
PricingVariant                  float64
PayFreq                           int32
AnniversaryDt            datetime64[ns]
IndexationPercent                 int64
IndexationStartDt        datetime64[ns]
PremHolStartDt           datetime64[ns]
PremHolEndDt             datetime64[ns]
Premium                         float64
PremiumCalcMethod                object
ServicingBroker                  object
InitialBroker                    object
BrokerCategory                   object
InitialRate                     float64
RenewalRate                     float

,ContractNumber,ProdCat,ProdCode,PolicyStatus,CoverEndDt,SnapshotDt,Switch_Flag,PolicyFeeType,PricingVariant,PayFreq,...,EUSanctioned,AddressDistrict,Postal_Code,Postal_Code_Name,District,State,VN_MILIEU_CODE,VN_TARGET_GROUP_CODE,VN_MILIEU_DESCRIPTION,VP_MILIEU_DESCRIPTION
0,6074160W,XRPP,XRPP_2005V1,I,2028-12-01,2020-01-31,0,Ein,NaN,12,...,False,0,52072.0,Aachen,St�dteregion Aachen,North Rhine-Westphalia,NaN,NaN,NaN,NaN
1,6074937L,XRPP,XRPP_2005V1,I,2033-12-01,2020-01-31,0,Ein,NaN,12,...,False,0,10823.0,Berlin,Berlin,Berlin,NaN,NaN,NaN,NaN
2,6074962M,XRPP,XRPP_2005V1,I,2042-12-01,2020-01-31,0,Ein,NaN,12,...,False,0,61440.0,Oberursel (Taunus),Landkreis Hochtaunuskreis,Hesse,NaN,NaN,NaN,NaN
3,6075850C,XRPP,XRPP_2005V1,F,2034-12-01,2020-01-31,0,Ein,NaN,12,...,False,0,42719.0,Solingen,Solingen,North Rhine-Westphalia,NaN,NaN,NaN,NaN
4,6100638P,XRPP,XRPP_2005V1,I,2046-04-01,2020-01-31,0,Ein,NaN,12,...,False,0,40789.0,Monheim am Rhein,Landkreis Mettmann,North Rhine-Westphalia,NaN,NaN,NaN,NaN


## Create the age and term columns.

In [102]:
# Set all dates to proper dates
#df_Horizon_All_Features['CoverEndDt'] = pd.to_datetime(df_Horizon_All_Features['CoverEndDt'])
#df_Horizon_All_Features['SnapshotDt'] = pd.to_datetime(df_Horizon_All_Features['SnapshotDt'])
#df_Horizon_All_Features['RiskCommDt'] = pd.to_datetime(df_Horizon_All_Features['RiskCommDt'])
#df_Horizon_All_Features['AnniversaryDt'] = pd.to_datetime(df_Horizon_All_Features['AnniversaryDt'])
#df_Horizon_All_Features['DOB'] = pd.to_datetime(df_Horizon_All_Features['DOB'])

# Calculate the term columns
df_Horizon_All_Features['Term'] = ((df_Horizon_All_Features.CoverEndDt - df_Horizon_All_Features.RiskCommDt)/np.timedelta64(1, 'Y')).apply(lambda x: round(x, 1))
df_Horizon_All_Features['Term_Passed'] = ((df_Horizon_All_Features.SnapshotDt - df_Horizon_All_Features.RiskCommDt)/np.timedelta64(1, 'Y')).apply(lambda x: round(x, 1))
df_Horizon_All_Features['Term_Remaining'] = ((df_Horizon_All_Features.CoverEndDt - df_Horizon_All_Features.SnapshotDt)/np.timedelta64(1, 'Y')).apply(lambda x: round(x, 1))

# Set a flag for policies with less or equal to 5 and less than or equal to 10 years term remaining.
df_Horizon_All_Features['TermRemainingLTEQ_5'] = '0'
df_Horizon_All_Features['TermRemainingLTEQ_10'] = '0'
df_Horizon_All_Features.loc[df_Horizon_All_Features['Term_Remaining'] <= 5, 'TermRemainingLTEQ_5'] = '1'
df_Horizon_All_Features.loc[df_Horizon_All_Features['Term_Remaining'] <= 10, 'TermRemainingLTEQ_10'] = '1'

# Calculate teh age at each snapshot
df_Horizon_All_Features['Age'] = ((df_Horizon_All_Features.SnapshotDt - df_Horizon_All_Features.DOB)/np.timedelta64(1, 'Y'))

# Calculate the anniversary month
df_Horizon_All_Features['AnniversaryMth'] = df_Horizon_All_Features['AnniversaryDt'].dt.month

print(df_Horizon_All_Features.dtypes)
df_Horizon_All_Features.head(5)

ContractNumber                   object
ProdCat                          object
ProdCode                         object
PolicyStatus                     object
CoverEndDt               datetime64[ns]
SnapshotDt               datetime64[ns]
Switch_Flag                       int32
PolicyFeeType                    object
PricingVariant                  float64
PayFreq                           int32
AnniversaryDt            datetime64[ns]
IndexationPercent                 int64
IndexationStartDt        datetime64[ns]
PremHolStartDt           datetime64[ns]
PremHolEndDt             datetime64[ns]
Premium                         float64
PremiumCalcMethod                object
ServicingBroker                  object
InitialBroker                    object
BrokerCategory                   object
InitialRate                     float64
RenewalRate                     float64
RiskCommDt               datetime64[ns]
VIPType                          object
Nationality                      object


,ContractNumber,ProdCat,ProdCode,PolicyStatus,CoverEndDt,SnapshotDt,Switch_Flag,PolicyFeeType,PricingVariant,PayFreq,...,VN_TARGET_GROUP_CODE,VN_MILIEU_DESCRIPTION,VP_MILIEU_DESCRIPTION,Term,Term_Passed,Term_Remaining,TermRemainingLTEQ_5,TermRemainingLTEQ_10,Age,AnniversaryMth
0,6074160W,XRPP,XRPP_2005V1,I,2028-12-01,2020-01-31,0,Ein,NaN,12,...,NaN,NaN,NaN,21.0,12.2,8.8,0,1,57.003224,12.0
1,6074937L,XRPP,XRPP_2005V1,I,2033-12-01,2020-01-31,0,Ein,NaN,12,...,NaN,NaN,NaN,27.0,13.2,13.8,0,0,51.560265,12.0
2,6074962M,XRPP,XRPP_2005V1,I,2042-12-01,2020-01-31,0,Ein,NaN,12,...,NaN,NaN,NaN,36.0,13.2,22.8,0,0,42.492317,12.0
3,6075850C,XRPP,XRPP_2005V1,F,2034-12-01,2020-01-31,0,Ein,NaN,12,...,NaN,NaN,NaN,28.0,13.2,14.8,0,0,45.996838,12.0
4,6100638P,XRPP,XRPP_2005V1,I,2046-04-01,2020-01-31,0,Ein,NaN,12,...,NaN,NaN,NaN,39.0,12.8,26.2,0,0,39.653107,4.0


## Drop all non predictive features and write to a csv file.

In [103]:
# Delete obsolete or PII columns
df_Horizon_All_Features = df_Horizon_All_Features.drop(['CoverEndDt','AnniversaryDt','PremHolStartDt','PremHolEndDt','IndexationStartDt','RiskCommDt','DOB'],axis=1)

print(df_Horizon_All_Features.dtypes)

ContractNumber                   object
ProdCat                          object
ProdCode                         object
PolicyStatus                     object
SnapshotDt               datetime64[ns]
Switch_Flag                       int32
PolicyFeeType                    object
PricingVariant                  float64
PayFreq                           int32
IndexationPercent                 int64
Premium                         float64
PremiumCalcMethod                object
ServicingBroker                  object
InitialBroker                    object
BrokerCategory                   object
InitialRate                     float64
RenewalRate                     float64
VIPType                          object
Nationality                      object
IsSmoker                         object
PoliticallyExposed               object
BirthPlace                       object
Gender                           object
MaritalStatus                    object
PensionDisability               float64


In [104]:
df_Horizon_All_Features.isna().any()

ContractNumber           False
ProdCat                  False
ProdCode                 False
PolicyStatus             False
SnapshotDt               False
Switch_Flag              False
PolicyFeeType            False
PricingVariant            True
PayFreq                  False
IndexationPercent        False
Premium                  False
PremiumCalcMethod        False
ServicingBroker           True
InitialBroker             True
BrokerCategory            True
InitialRate               True
RenewalRate               True
VIPType                   True
Nationality               True
IsSmoker                  True
PoliticallyExposed        True
BirthPlace                True
Gender                    True
MaritalStatus             True
PensionDisability         True
WOPDisability             True
AddressCity               True
PartnerType               True
EUSanctioned              True
AddressDistrict           True
Postal_Code               True
Postal_Code_Name          True
District

In [108]:
df_Horizon_All_Features.isna().any()

ContractNumber           False
ProdCat                  False
ProdCode                 False
PolicyStatus             False
SnapshotDt               False
Switch_Flag              False
PolicyFeeType            False
PricingVariant           False
PayFreq                  False
IndexationPercent        False
Premium                  False
PremiumCalcMethod        False
ServicingBroker          False
InitialBroker            False
BrokerCategory           False
InitialRate              False
RenewalRate              False
VIPType                  False
Nationality              False
IsSmoker                 False
PoliticallyExposed       False
BirthPlace               False
Gender                   False
MaritalStatus            False
PensionDisability        False
WOPDisability            False
AddressCity              False
PartnerType              False
EUSanctioned             False
AddressDistrict          False
Postal_Code              False
Postal_Code_Name         False
District

In [106]:
df_Horizon_All_Features['Postal_Code'] = df_Horizon_All_Features['Postal_Code'].fillna(0)
df_Horizon_All_Features['PricingVariant'] = df_Horizon_All_Features['PricingVariant'].fillna(0)
df_Horizon_All_Features['ServicingBroker'] = df_Horizon_All_Features['ServicingBroker'].fillna('')
df_Horizon_All_Features['InitialBroker'] = df_Horizon_All_Features['InitialBroker'].fillna('')
df_Horizon_All_Features['BrokerCategory'] = df_Horizon_All_Features['BrokerCategory'].fillna('')
df_Horizon_All_Features['InitialRate'] = df_Horizon_All_Features['InitialRate'].fillna(0)
df_Horizon_All_Features['RenewalRate'] = df_Horizon_All_Features['RenewalRate'].fillna(0)
df_Horizon_All_Features['Term'] = df_Horizon_All_Features['Term'].fillna(0)
df_Horizon_All_Features['Term_Passed'] = df_Horizon_All_Features['Term_Passed'].fillna(0)
df_Horizon_All_Features['Term_Remaining'] = df_Horizon_All_Features['Term_Remaining'].fillna(0)
df_Horizon_All_Features['PensionDisability'] = df_Horizon_All_Features['PensionDisability'].fillna(0)
df_Horizon_All_Features['WOPDisability'] = df_Horizon_All_Features['WOPDisability'].fillna(0)
df_Horizon_All_Features['VIPType'] = df_Horizon_All_Features['VIPType'].fillna('')
df_Horizon_All_Features['Nationality'] = df_Horizon_All_Features['Nationality'].fillna('')
df_Horizon_All_Features['IsSmoker'] = df_Horizon_All_Features['IsSmoker'].fillna(0)
df_Horizon_All_Features['PoliticallyExposed'] = df_Horizon_All_Features['PoliticallyExposed'].fillna('')
df_Horizon_All_Features['BirthPlace'] = df_Horizon_All_Features['BirthPlace'].fillna('')
df_Horizon_All_Features['Gender'] = df_Horizon_All_Features['Gender'].fillna('')
df_Horizon_All_Features['MaritalStatus'] = df_Horizon_All_Features['MaritalStatus'].fillna('')
df_Horizon_All_Features['AddressCity'] = df_Horizon_All_Features['AddressCity'].fillna('')
df_Horizon_All_Features['PartnerType'] = df_Horizon_All_Features['PartnerType'].fillna('')
df_Horizon_All_Features['EUSanctioned'] = df_Horizon_All_Features['EUSanctioned'].fillna('')
df_Horizon_All_Features['AddressDistrict'] = df_Horizon_All_Features['AddressDistrict'].fillna('')
df_Horizon_All_Features['Postal_Code_Name'] = df_Horizon_All_Features['Postal_Code_Name'].fillna('')
df_Horizon_All_Features['District'] = df_Horizon_All_Features['District'].fillna('')
df_Horizon_All_Features['State'] = df_Horizon_All_Features['State'].fillna('')
df_Horizon_All_Features['VN_MILIEU_CODE'] = df_Horizon_All_Features['VN_MILIEU_CODE'].fillna('')
df_Horizon_All_Features['VN_TARGET_GROUP_CODE'] = df_Horizon_All_Features['VN_TARGET_GROUP_CODE'].fillna('')
df_Horizon_All_Features['VN_MILIEU_DESCRIPTION'] = df_Horizon_All_Features['VN_MILIEU_DESCRIPTION'].fillna('')
df_Horizon_All_Features['VP_MILIEU_DESCRIPTION'] = df_Horizon_All_Features['VP_MILIEU_DESCRIPTION'].fillna('')
df_Horizon_All_Features['AnniversaryMth'] = df_Horizon_All_Features['AnniversaryMth'].fillna(0)
df_Horizon_All_Features['Age'] = df_Horizon_All_Features['Age'].fillna(0)

df_Horizon_All_Features.isna().any()

ContractNumber           False
ProdCat                  False
ProdCode                 False
PolicyStatus             False
SnapshotDt               False
Switch_Flag              False
PolicyFeeType            False
PricingVariant           False
PayFreq                  False
IndexationPercent        False
Premium                  False
PremiumCalcMethod        False
ServicingBroker          False
InitialBroker            False
BrokerCategory           False
InitialRate              False
RenewalRate              False
VIPType                  False
Nationality              False
IsSmoker                 False
PoliticallyExposed       False
BirthPlace               False
Gender                   False
MaritalStatus            False
PensionDisability        False
WOPDisability            False
AddressCity              False
PartnerType              False
EUSanctioned             False
AddressDistrict          False
Postal_Code               True
Postal_Code_Name         False
District

In [109]:
df_Horizon_All_Features.isna().any()

ContractNumber           False
ProdCat                  False
ProdCode                 False
PolicyStatus             False
SnapshotDt               False
Switch_Flag              False
PolicyFeeType            False
PricingVariant           False
PayFreq                  False
IndexationPercent        False
Premium                  False
PremiumCalcMethod        False
ServicingBroker          False
InitialBroker            False
BrokerCategory           False
InitialRate              False
RenewalRate              False
VIPType                  False
Nationality              False
IsSmoker                 False
PoliticallyExposed       False
BirthPlace               False
Gender                   False
MaritalStatus            False
PensionDisability        False
WOPDisability            False
AddressCity              False
PartnerType              False
EUSanctioned             False
AddressDistrict          False
Postal_Code              False
Postal_Code_Name         False
District

In [110]:
# Write out to a comma separated values file.
# giving directory name
filename = input_path + '\Full_Horizon_All_Features.csv'
df_Horizon_All_Features.to_csv(filename, encoding='utf-8', index=False)

print(df_Horizon_All_Features.dtypes)
df_Horizon_All_Features.head(10)

ContractNumber                   object
ProdCat                          object
ProdCode                         object
PolicyStatus                     object
SnapshotDt               datetime64[ns]
Switch_Flag                       int32
PolicyFeeType                    object
PricingVariant                  float64
PayFreq                           int32
IndexationPercent                 int64
Premium                         float64
PremiumCalcMethod                object
ServicingBroker                  object
InitialBroker                    object
BrokerCategory                   object
InitialRate                     float64
RenewalRate                     float64
VIPType                          object
Nationality                      object
IsSmoker                         object
PoliticallyExposed               object
BirthPlace                       object
Gender                           object
MaritalStatus                    object
PensionDisability               float64


,ContractNumber,ProdCat,ProdCode,PolicyStatus,SnapshotDt,Switch_Flag,PolicyFeeType,PricingVariant,PayFreq,IndexationPercent,...,VN_TARGET_GROUP_CODE,VN_MILIEU_DESCRIPTION,VP_MILIEU_DESCRIPTION,Term,Term_Passed,Term_Remaining,TermRemainingLTEQ_5,TermRemainingLTEQ_10,Age,AnniversaryMth
0,6074160W,XRPP,XRPP_2005V1,I,2020-01-31,0,Ein,0.0,12,0,...,,,,21.0,12.2,8.8,0,1,57.003224,12.0
1,6074937L,XRPP,XRPP_2005V1,I,2020-01-31,0,Ein,0.0,12,0,...,,,,27.0,13.2,13.8,0,0,51.560265,12.0
2,6074962M,XRPP,XRPP_2005V1,I,2020-01-31,0,Ein,0.0,12,0,...,,,,36.0,13.2,22.8,0,0,42.492317,12.0
3,6075850C,XRPP,XRPP_2005V1,F,2020-01-31,0,Ein,0.0,12,0,...,,,,28.0,13.2,14.8,0,0,45.996838,12.0
4,6100638P,XRPP,XRPP_2005V1,I,2020-01-31,0,Ein,0.0,12,0,...,,,,39.0,12.8,26.2,0,0,39.653107,4.0
5,6100746E,XRPP,XRPP_2005V1,I,2020-01-31,0,Ein,0.0,12,0,...,,,,37.0,13.8,23.2,0,0,42.391014,5.0
6,6214246Y,XRPP,XRPP_2005V1,I,2020-01-31,0,Ein,0.0,12,0,...,,,,29.0,14.3,14.7,0,0,50.558191,10.0
7,6215630F,XRPP,XRPP_2005V1,I,2020-01-31,0,Ein,0.0,12,0,...,,,,37.0,14.2,22.8,0,0,42.642902,12.0
8,6215740Y,XRPP,XRPP_2005V1,I,2020-01-31,0,Ein,0.0,12,0,...,,,,29.0,14.3,14.7,0,0,51.141365,10.0
9,6215812A,XRPP,XRPP_2005V1,I,2020-01-31,0,Ein,4015.0,12,0,...,,,,40.0,14.3,25.7,0,0,39.957015,10.0


# Clean up dataframes to release memory.

In [111]:
%who_ls DataFrame 

['df_Horizon_All_Features']

In [112]:
del df_Horizon_All_Features

print('==========================')
now = datetime.now()
print(now)
print('==========================')
print(psutil.virtual_memory())
print('==========================')


2024-07-20 17:09:59.699497
svmem(total=16756752384, available=5794394112, percent=65.4, used=10962358272, free=5794394112)
